# 🚀 Optimized Bidirectional LSTM for Order Prediction
## Focus: BiLSTM with Comprehensive Model Management

### Current Performance (Your Results):
- ✅ **MAE: 3.59 days** (Excellent! Much better than baseline 6.17)
- ✅ **Within 7 days: 91.9%** (Outstanding accuracy)
- ✅ **Within 14 days: 96.3%** (Very high precision)

### This Notebook Includes:
1. **Enhanced BiLSTM Architecture** (building on your success)
2. **Comprehensive Model Saving** (multiple formats)
3. **Model Versioning & Checkpointing**
4. **Easy Model Loading & Inference**
5. **Performance Monitoring & Logging**

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn scikit-learn tensorflow keras -q

print("✅ All packages installed successfully!")

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import json
import pickle

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    LSTM, Bidirectional, Dense, Dropout, 
    BatchNormalization, Attention
)
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, 
    ModelCheckpoint, CSVLogger, TensorBoard
)
from tensorflow.keras.regularizers import l1_l2

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')

# Reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print("✅ Libraries imported successfully!")
print(f"📦 TensorFlow version: {tf.__version__}")
print(f"🎲 Random seed set to: {RANDOM_SEED}")

## 🔧 Configuration & Model Directories

In [ ]:
# Create directory structure for model management
MODEL_DIR = 'models'
CHECKPOINT_DIR = os.path.join(MODEL_DIR, 'checkpoints')
LOGS_DIR = os.path.join(MODEL_DIR, 'logs')
SCALER_DIR = os.path.join(MODEL_DIR, 'scalers')
METADATA_DIR = os.path.join(MODEL_DIR, 'metadata')

# Create directories
for directory in [MODEL_DIR, CHECKPOINT_DIR, LOGS_DIR, SCALER_DIR, METADATA_DIR]:
    os.makedirs(directory, exist_ok=True)

# Configuration
CONFIG = {
    'model_name': 'bilstm_order_prediction',
    'version': '1.0',
    'random_seed': RANDOM_SEED,
    'test_size': 0.2,
    'validation_split': 0.15,
    'batch_size': 64,
    'epochs': 150,
    'learning_rate': 0.001,
    'patience_early_stop': 20,
    'patience_reduce_lr': 10,
}

print("✅ Directory structure created:")
print(f"   📁 Models: {MODEL_DIR}")
print(f"   📁 Checkpoints: {CHECKPOINT_DIR}")
print(f"   📁 Logs: {LOGS_DIR}")
print(f"   📁 Scalers: {SCALER_DIR}")
print(f"   📁 Metadata: {METADATA_DIR}")
print(f"\n⚙️  Configuration loaded")

## 🎯 Enhanced Feature Engineering
### Based on your successful implementation + additional improvements

In [ ]:
def create_enhanced_features(df):
    """
    Create enhanced features optimized for BiLSTM
    Based on your successful implementation (3.59 MAE)
    """
    df = df.copy()
    
    # Convert dates
    df['sales_order_creation_date'] = pd.to_datetime(df['sales_order_creation_date'])
    df['actual_expedition_date'] = pd.to_datetime(df['actual_expedition_date'])
    
    # Sort by client and ORDER CREATION date (when client placed order)
    # This is crucial - we predict when they'll PLACE next order, not when it delivers
    df = df.sort_values(['client_name', 'sales_order_creation_date'])
    
    print("Creating features...")
    
    # ========================================================================
    # 1. CORE TEMPORAL FEATURES (Most Important)
    # ========================================================================
    
    # Days since last order
    df['days_since_last_order'] = df.groupby('client_name')['sales_order_creation_date'].diff().dt.days
    
    # Historical statistics
    df['days_since_last_order_mean'] = df.groupby('client_name')['days_since_last_order'].transform(
        lambda x: x.expanding().mean()
    )
    df['days_since_last_order_std'] = df.groupby('client_name')['days_since_last_order'].transform(
        lambda x: x.expanding().std()
    )
    df['days_since_last_order_min'] = df.groupby('client_name')['days_since_last_order'].transform(
        lambda x: x.expanding().min()
    )
    df['days_since_last_order_max'] = df.groupby('client_name')['days_since_last_order'].transform(
        lambda x: x.expanding().max()
    )
    
    # ========================================================================
    # 2. CLIENT BEHAVIOR METRICS
    # ========================================================================z
    
    # Order sequence and frequency
    df['order_sequence'] = df.groupby('client_name').cumcount() + 1
    # Use ORDER CREATION dates for client lifetime (reordering timeline)
    df['client_lifetime_days'] = df.groupby('client_name')['sales_order_creation_date'].transform(
        lambda x: (x - x.min()).dt.days
    )
    df['order_frequency_per_month'] = (df['order_sequence'] / (df['client_lifetime_days'] / 30)).replace(
        [np.inf, -np.inf], 0
    )
    
    # Order counts
    df['total_amount_delivered_tm_count'] = df.groupby('client_name').cumcount() + 1
    
    # ========================================================================
    # 3. ROLLING WINDOW FEATURES (Multiple Horizons)
    # ========================================================================
    
    for window in [3, 5, 7, 10]:
        # Days between orders
        df[f'rolling_avg_days_{window}'] = df.groupby('client_name')['days_since_last_order'].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        df[f'rolling_std_days_{window}'] = df.groupby('client_name')['days_since_last_order'].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )
        
        # Quantity patterns
        df[f'rolling_avg_quantity_{window}'] = df.groupby('client_name')['total_amount_delivered_tm'].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        df[f'rolling_std_quantity_{window}'] = df.groupby('client_name')['total_amount_delivered_tm'].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )
    
    # ========================================================================
    # 4. TREND & MOMENTUM FEATURES
    # ========================================================================
    
    # Ordering frequency trend
    df['order_frequency_trend'] = df.groupby('client_name')['order_frequency_per_month'].transform(
        lambda x: x.diff()
    )
    
    # Days since last order trend
    df['days_trend'] = df.groupby('client_name')['days_since_last_order'].transform(
        lambda x: x.diff()
    )
    
    # Quantity trend
    df['quantity_trend'] = df.groupby('client_name')['total_amount_delivered_tm'].transform(
        lambda x: x.diff()
    )
    
    # Recent vs historical comparison
    df['recent_vs_historical_frequency'] = df['rolling_avg_days_3'] / (df['days_since_last_order_mean'] + 1)
    
    # ========================================================================
    # 5. CLIENT MATURITY & CONSISTENCY
    # ========================================================================
    
    # Maturity level
    df['client_maturity'] = pd.cut(
        df['total_amount_delivered_tm_count'],
        bins=[0, 5, 20, np.inf],
        labels=[0, 1, 2]  # 0=new, 1=established, 2=veteran
    ).astype(int)
    
    # Consistency score (inverse of coefficient of variation)
    df['ordering_consistency'] = df['days_since_last_order_mean'] / (df['days_since_last_order_std'] + 1)
    
    # High frequency flag
    df['is_high_frequency'] = (df['order_frequency_per_month'] > df['order_frequency_per_month'].median()).astype(int)
    
    # ========================================================================
    # 6. QUANTITY FEATURES
    # ========================================================================
    
    df['total_amount_delivered_tm_mean'] = df.groupby('client_name')['total_amount_delivered_tm'].transform(
        lambda x: x.expanding().mean()
    )
    df['total_amount_delivered_tm_std'] = df.groupby('client_name')['total_amount_delivered_tm'].transform(
        lambda x: x.expanding().std()
    )
    
    # ========================================================================
    # 7. MINIMAL TEMPORAL CONTEXT (Lower priority but keep for context)
    # ========================================================================
    # Use ORDER CREATION dates (when client placed order, not delivery date)
    df['order_month'] = df['sales_order_creation_date'].dt.month
    df['order_quarter'] = df['sales_order_creation_date'].dt.quarter
    df['order_day_of_month'] = df['sales_order_creation_date'].dt.day
    
    # ========================================================================
    # 8. TARGET VARIABLE
    # ========================================================================
    
    df['days_until_next_order'] = df.groupby('client_name')['days_since_last_order'].shift(-1)
    
    # Fill missing values
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(0)
    
    print(f"✅ Feature engineering complete!")
    print(f"   Total features created: {len(numeric_cols)}")
    
    return df

print("✅ Feature engineering function defined")

## 🎯 Intelligent Feature Selection

In [ ]:
def select_best_features():
    """
    Select features optimized for BiLSTM based on importance analysis
    """
    
    # HIGH PRIORITY - Core temporal features
    core_features = [
        'days_since_last_order',
        'days_since_last_order_mean',
        'days_since_last_order_std',
        'days_since_last_order_min',
        'days_since_last_order_max',
    ]
    
    # HIGH PRIORITY - Client behavior
    behavior_features = [
        'order_frequency_per_month',
        'total_amount_delivered_tm_count',
        'order_sequence',
        'client_lifetime_days',
    ]
    
    # MEDIUM PRIORITY - Rolling windows (multiple horizons)
    rolling_features = []
    for window in [3, 5, 7, 10]:
        rolling_features.extend([
            f'rolling_avg_days_{window}',
            f'rolling_std_days_{window}',
            f'rolling_avg_quantity_{window}',
            f'rolling_std_quantity_{window}',
        ])
    
    # MEDIUM PRIORITY - Trends
    trend_features = [
        'order_frequency_trend',
        'days_trend',
        'quantity_trend',
        'recent_vs_historical_frequency',
    ]
    
    # LOW PRIORITY - Consistency & maturity
    meta_features = [
        'client_maturity',
        'ordering_consistency',
        'is_high_frequency',
    ]
    
    # LOW PRIORITY - Quantity
    quantity_features = [
        'total_amount_delivered_tm',
        'total_amount_delivered_tm_mean',
        'total_amount_delivered_tm_std',
    ]
    
    # MINIMAL - Temporal context
    temporal_features = [
        'order_month',
        'order_quarter',
        'order_day_of_month',
    ]
    
    # Combine all
    selected_features = (
        core_features + 
        behavior_features + 
        rolling_features + 
        trend_features + 
        meta_features + 
        quantity_features + 
        temporal_features
    )
    
    print(f"\n📊 Feature Selection Summary:")
    print(f"   Core Temporal: {len(core_features)}")
    print(f"   Behavior: {len(behavior_features)}")
    print(f"   Rolling Windows: {len(rolling_features)}")
    print(f"   Trends: {len(trend_features)}")
    print(f"   Meta: {len(meta_features)}")
    print(f"   Quantity: {len(quantity_features)}")
    print(f"   Temporal: {len(temporal_features)}")
    print(f"   " + "="*50)
    print(f"   Total Selected: {len(selected_features)}")
    
    return selected_features

print("✅ Feature selection function defined")

## 🔧 Data Preparation with Model Persistence

In [ ]:
def prepare_data_for_lstm(df, selected_features, config):
    """
    Prepare data for LSTM with scaler persistence
    """
    
    # Remove rows without target
    df_clean = df.dropna(subset=['days_until_next_order']).copy()
    
    # Separate features and target
    X = df_clean[selected_features].copy()
    y = df_clean['days_until_next_order'].copy()
    
    # Split data (temporal split - no shuffle)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=config['test_size'], 
        random_state=config['random_seed'],
        shuffle=False  # Preserve temporal order
    )
    
    # Scale features using RobustScaler (better for outliers)
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Reshape for LSTM: (samples, timesteps, features)
    X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))
    
    # Save scaler
    scaler_path = os.path.join(SCALER_DIR, 'feature_scaler.pkl')
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    print(f"\n💾 Scaler saved to: {scaler_path}")
    
    # Save feature names
    feature_path = os.path.join(METADATA_DIR, 'feature_names.json')
    with open(feature_path, 'w') as f:
        json.dump(selected_features, f, indent=2)
    print(f"💾 Feature names saved to: {feature_path}")
    
    print(f"\n📊 Data Preparation Summary:")
    print(f"   Training samples: {X_train_lstm.shape[0]:,}")
    print(f"   Test samples: {X_test_lstm.shape[0]:,}")
    print(f"   Number of features: {X_train_lstm.shape[2]}")
    print(f"   LSTM input shape: {X_train_lstm.shape}")
    print(f"   Target mean (train): {y_train.mean():.2f} days")
    print(f"   Target std (train): {y_train.std():.2f} days")
    
    return {
        'X_train': X_train_lstm,
        'X_test': X_test_lstm,
        'y_train': y_train,
        'y_test': y_test,
        'scaler': scaler,
        'feature_names': selected_features
    }

print("✅ Data preparation function defined")

## 🧠 Enhanced Bidirectional LSTM Architecture

In [ ]:
def build_enhanced_bilstm(input_shape, config):
    """
    Build enhanced BiLSTM based on your successful architecture
    Optimized for 3.59 MAE performance
    """
    
    model = Sequential([
        # First Bidirectional LSTM layer with attention
        Bidirectional(
            LSTM(
                128,
                return_sequences=True,
                kernel_regularizer=l1_l2(l1=0.001, l2=0.001),
                recurrent_dropout=0.1
            ),
            input_shape=input_shape
        ),
        BatchNormalization(),
        Dropout(0.3),
        
        # Second Bidirectional LSTM layer
        Bidirectional(
            LSTM(
                64,
                return_sequences=False,
                kernel_regularizer=l1_l2(l1=0.001, l2=0.001),
                recurrent_dropout=0.1
            )
        ),
        BatchNormalization(),
        Dropout(0.3),
        
        # Dense layers
        Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.0005, l2=0.0005)),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(32, activation='relu'),
        Dropout(0.2),
        
        Dense(16, activation='relu'),
        
        # Output layer
        Dense(1, activation='linear')
    ])
    
    # Compile with Huber loss (robust to outliers)
    optimizer = keras.optimizers.Adam(
        learning_rate=config['learning_rate'],
        clipnorm=1.0  # Gradient clipping
    )
    
    model.compile(
        optimizer=optimizer,
        loss='huber',  # Robust to outliers
        metrics=['mae', 'mse']
    )
    
    return model

print("✅ Model architecture function defined")

## 📊 Comprehensive Training Callbacks

In [ ]:
def create_callbacks(config):
    """
    Create comprehensive callbacks for training
    """
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    callbacks = [
        # 1. Early Stopping
        EarlyStopping(
            monitor='val_loss',
            patience=config['patience_early_stop'],
            restore_best_weights=True,
            verbose=1,
            mode='min'
        ),
        
        # 2. Reduce Learning Rate on Plateau
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=config['patience_reduce_lr'],
            min_lr=1e-7,
            verbose=1,
            mode='min'
        ),
        
        # 3. Model Checkpoint (save best model)
        ModelCheckpoint(
            filepath=os.path.join(CHECKPOINT_DIR, f'best_model_{timestamp}.h5'),
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=False,
            verbose=1,
            mode='min'
        ),
        
        # 4. CSV Logger (detailed training history)
        CSVLogger(
            filename=os.path.join(LOGS_DIR, f'training_log_{timestamp}.csv'),
            append=False
        ),
        
        # 5. TensorBoard (optional, for visualization)
        TensorBoard(
            log_dir=os.path.join(LOGS_DIR, f'tensorboard_{timestamp}'),
            histogram_freq=1,
            write_graph=True
        )
    ]
    
    print(f"\n📋 Callbacks configured:")
    print(f"   ✓ Early Stopping (patience={config['patience_early_stop']})")
    print(f"   ✓ Reduce LR on Plateau (patience={config['patience_reduce_lr']})")
    print(f"   ✓ Model Checkpoint")
    print(f"   ✓ CSV Logger")
    print(f"   ✓ TensorBoard")
    
    return callbacks, timestamp

print("✅ Callbacks function defined")

## 🚀 Model Training with Progress Tracking

In [ ]:
def train_bilstm(model, data, config, callbacks):
    """
    Train BiLSTM model with comprehensive monitoring
    """
    
    print("\n" + "="*80)
    print("🚀 STARTING MODEL TRAINING")
    print("="*80)
    
    training_start = datetime.now()
    
    history = model.fit(
        data['X_train'], data['y_train'],
        validation_split=config['validation_split'],
        epochs=config['epochs'],
        batch_size=config['batch_size'],
        callbacks=callbacks,
        verbose=1
    )
    
    training_duration = datetime.now() - training_start
    
    print("\n" + "="*80)
    print("✅ TRAINING COMPLETE")
    print("="*80)
    print(f"   Duration: {training_duration}")
    print(f"   Epochs completed: {len(history.history['loss'])}")
    print(f"   Best val_loss: {min(history.history['val_loss']):.4f}")
    print(f"   Best val_mae: {min(history.history['val_mae']):.4f}")
    
    return model, history

print("✅ Training function defined")

In [ ]:
# FIXED VERSION - Use this instead of the previous save_complete_model function
def save_complete_model(model, data, config, history, timestamp):
    """
    Save model in multiple formats with metadata (Keras 3 compatible)
    """
    
    print("\n" + "="*80)
    print("💾 SAVING MODEL")
    print("="*80)
    
    model_name = f"{config['model_name']}_v{config['version']}_{timestamp}"
    
    # 1. Save full model (.h5 format)
    h5_path = os.path.join(MODEL_DIR, f'{model_name}.h5')
    model.save(h5_path)
    print(f"✅ Model saved (HDF5): {h5_path}")
    
    # 2. Save model in native Keras format (Keras 3 format)
    keras_path = os.path.join(MODEL_DIR, f'{model_name}.keras')
    model.save(keras_path)
    print(f"✅ Model saved (Keras format): {keras_path}")
    
    # 3. Save model weights only
    weights_path = os.path.join(MODEL_DIR, f'{model_name}_weights.h5')
    model.save_weights(weights_path)
    print(f"✅ Weights saved: {weights_path}")
    
    # 4. Save model architecture (JSON)
    architecture_path = os.path.join(METADATA_DIR, f'{model_name}_architecture.json')
    with open(architecture_path, 'w') as f:
        f.write(model.to_json())
    print(f"✅ Architecture saved: {architecture_path}")
    
    # 5. Save training history
    history_path = os.path.join(METADATA_DIR, f'{model_name}_history.pkl')
    with open(history_path, 'wb') as f:
        pickle.dump(history.history, f)
    print(f"✅ Training history saved: {history_path}")
    
    # 6. Save metadata (configuration + performance)
    metadata = {
        'model_name': model_name,
        'timestamp': timestamp,
        'config': config,
        'training_info': {
            'total_epochs': len(history.history['loss']),
            'best_val_loss': float(min(history.history['val_loss'])),
            'best_val_mae': float(min(history.history['val_mae'])),
            'final_train_loss': float(history.history['loss'][-1]),
            'final_train_mae': float(history.history['mae'][-1]),
        },
        'data_info': {
            'n_features': len(data['feature_names']),
            'n_train_samples': int(data['X_train'].shape[0]),
            'n_test_samples': int(data['X_test'].shape[0]),
            'target_mean': float(data['y_train'].mean()),
            'target_std': float(data['y_train'].std()),
        }
    }
    
    metadata_path = os.path.join(METADATA_DIR, f'{model_name}_metadata.json')
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Metadata saved: {metadata_path}")
    
    # 7. Create a "latest" symlink or copy
    latest_path = os.path.join(MODEL_DIR, 'latest_model.h5')
    import shutil
    shutil.copy2(h5_path, latest_path)
    print(f"✅ Latest model link created: {latest_path}")
    
    print("\n" + "="*80)
    print("🎉 MODEL SAVED SUCCESSFULLY")
    print("="*80)
    print(f"\n📦 Saved artifacts:")
    print(f"   1. Full model (.h5): {h5_path}")
    print(f"   2. Keras format (.keras): {keras_path}")
    print(f"   3. Weights only: {weights_path}")
    print(f"   4. Architecture (JSON): {architecture_path}")
    print(f"   5. Training history: {history_path}")
    print(f"   6. Metadata: {metadata_path}")
    print(f"   7. Latest model: {latest_path}")
    
    return model_name, metadata

print("✅ FIXED Model saving function defined - ready to use!")


## 💾 Comprehensive Model Saving

In [ ]:
def save_complete_model(model, data, config, history, timestamp):
    """
    Save model in multiple formats with metadata
    """
    
    print("\n" + "="*80)
    print("💾 SAVING MODEL")
    print("="*80)
    
    model_name = f"{config['model_name']}_v{config['version']}_{timestamp}"
    
    # 1. Save full model (.h5 format)
    h5_path = os.path.join(MODEL_DIR, f'{model_name}.h5')
    model.save(h5_path)
    print(f"✅ Model saved (HDF5): {h5_path}")
    
    # 2. Save model in SavedModel format (TensorFlow)
    savedmodel_path = os.path.join(MODEL_DIR, f'{model_name}_savedmodel')
    model.save(savedmodel_path, save_format='tf')
    print(f"✅ Model saved (SavedModel): {savedmodel_path}")
    
    # 3. Save model weights only
    weights_path = os.path.join(MODEL_DIR, f'{model_name}_weights.h5')
    model.save_weights(weights_path)
    print(f"✅ Weights saved: {weights_path}")
    
    # 4. Save model architecture (JSON)
    architecture_path = os.path.join(METADATA_DIR, f'{model_name}_architecture.json')
    with open(architecture_path, 'w') as f:
        f.write(model.to_json())
    print(f"✅ Architecture saved: {architecture_path}")
    
    # 5. Save training history
    history_path = os.path.join(METADATA_DIR, f'{model_name}_history.pkl')
    with open(history_path, 'wb') as f:
        pickle.dump(history.history, f)
    print(f"✅ Training history saved: {history_path}")
    
    # 6. Save metadata (configuration + performance)
    metadata = {
        'model_name': model_name,
        'timestamp': timestamp,
        'config': config,
        'training_info': {
            'total_epochs': len(history.history['loss']),
            'best_val_loss': float(min(history.history['val_loss'])),
            'best_val_mae': float(min(history.history['val_mae'])),
            'final_train_loss': float(history.history['loss'][-1]),
            'final_train_mae': float(history.history['mae'][-1]),
        },
        'data_info': {
            'n_features': len(data['feature_names']),
            'n_train_samples': int(data['X_train'].shape[0]),
            'n_test_samples': int(data['X_test'].shape[0]),
            'target_mean': float(data['y_train'].mean()),
            'target_std': float(data['y_train'].std()),
        }
    }
    
    metadata_path = os.path.join(METADATA_DIR, f'{model_name}_metadata.json')
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Metadata saved: {metadata_path}")
    
    # 7. Create a "latest" symlink or copy
    latest_path = os.path.join(MODEL_DIR, 'latest_model.h5')
    import shutil
    shutil.copy2(h5_path, latest_path)
    print(f"✅ Latest model link created: {latest_path}")
    
    print("\n" + "="*80)
    print("🎉 MODEL SAVED SUCCESSFULLY")
    print("="*80)
    print(f"\n📦 Saved artifacts:")
    print(f"   1. Full model (.h5): {h5_path}")
    print(f"   2. SavedModel format: {savedmodel_path}")
    print(f"   3. Weights only: {weights_path}")
    print(f"   4. Architecture (JSON): {architecture_path}")
    print(f"   5. Training history: {history_path}")
    print(f"   6. Metadata: {metadata_path}")
    print(f"   7. Latest model: {latest_path}")
    
    return model_name, metadata

print("✅ Model saving function defined")

## 📊 Model Evaluation & Performance Analysis

In [ ]:
def evaluate_model_comprehensive(model, X_test, y_test):
    """
    Comprehensive model evaluation
    """
    
    print("\n" + "="*80)
    print("📊 MODEL EVALUATION")
    print("="*80)
    
    # Make predictions
    y_pred = model.predict(X_test, verbose=0).flatten()
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    # Median absolute error
    median_ae = np.median(np.abs(y_test - y_pred))
    
    # Mean Absolute Percentage Error (handle division by zero)
    mape = np.mean(np.abs((y_test - y_pred) / np.where(y_test != 0, y_test, 1))) * 100
    
    # Prediction accuracy at different thresholds
    within_3_days = np.mean(np.abs(y_test - y_pred) <= 3) * 100
    within_7_days = np.mean(np.abs(y_test - y_pred) <= 7) * 100
    within_14_days = np.mean(np.abs(y_test - y_pred) <= 14) * 100
    within_30_days = np.mean(np.abs(y_test - y_pred) <= 30) * 100
    
    print(f"\n🎯 Core Metrics:")
    print(f"   MAE (Mean Absolute Error):      {mae:.2f} days")
    print(f"   RMSE (Root Mean Squared Error): {rmse:.2f} days")
    print(f"   R² Score:                       {r2:.4f}")
    print(f"   Median Absolute Error:          {median_ae:.2f} days")
    print(f"   MAPE:                           {mape:.2f}%")
    
    print(f"\n🎯 Prediction Accuracy:")
    print(f"   Within  3 days:  {within_3_days:5.1f}%")
    print(f"   Within  7 days:  {within_7_days:5.1f}%")
    print(f"   Within 14 days:  {within_14_days:5.1f}%")
    print(f"   Within 30 days:  {within_30_days:5.1f}%")
    
    # Error distribution
    errors = y_test - y_pred
    print(f"\n📈 Error Distribution:")
    print(f"   Mean error:      {errors.mean():6.2f} days")
    print(f"   Std error:       {errors.std():6.2f} days")
    print(f"   Min error:       {errors.min():6.2f} days")
    print(f"   25th percentile: {np.percentile(errors, 25):6.2f} days")
    print(f"   50th percentile: {np.percentile(errors, 50):6.2f} days")
    print(f"   75th percentile: {np.percentile(errors, 75):6.2f} days")
    print(f"   Max error:       {errors.max():6.2f} days")
    
    results = {
        'predictions': y_pred,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'median_ae': median_ae,
        'mape': mape,
        'within_3': within_3_days,
        'within_7': within_7_days,
        'within_14': within_14_days,
        'within_30': within_30_days,
        'errors': errors
    }
    
    return results

print("✅ Evaluation function defined")

## 📈 Visualization Functions

In [ ]:
def plot_training_history(history, save_path=None):
    """
    Plot training history
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 0].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # MAE
    axes[0, 1].plot(history.history['mae'], label='Train MAE', linewidth=2)
    axes[0, 1].plot(history.history['val_mae'], label='Val MAE', linewidth=2)
    axes[0, 1].set_title('Mean Absolute Error', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('MAE (days)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # MSE
    axes[1, 0].plot(history.history['mse'], label='Train MSE', linewidth=2)
    axes[1, 0].plot(history.history['val_mse'], label='Val MSE', linewidth=2)
    axes[1, 0].set_title('Mean Squared Error', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('MSE')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Learning Rate (if available)
    if 'lr' in history.history:
        axes[1, 1].plot(history.history['lr'], linewidth=2, color='red')
        axes[1, 1].set_title('Learning Rate', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Learning Rate')
        axes[1, 1].set_yscale('log')
        axes[1, 1].grid(True, alpha=0.3)
    else:
        axes[1, 1].axis('off')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Training history plot saved: {save_path}")
    
    plt.show()

def plot_predictions(y_test, y_pred, save_path=None):
    """
    Plot predictions vs actual
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Scatter plot
    axes[0].scatter(y_test, y_pred, alpha=0.5, s=20)
    axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                'r--', lw=2, label='Perfect Prediction')
    axes[0].set_xlabel('Actual Days', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Predicted Days', fontsize=12, fontweight='bold')
    axes[0].set_title('Predictions vs Actual', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Error distribution
    errors = y_test - y_pred
    axes[1].hist(errors, bins=50, edgecolor='black', alpha=0.7)
    axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2, label='Zero Error')
    axes[1].set_xlabel('Prediction Error (days)', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
    axes[1].set_title('Error Distribution', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Predictions plot saved: {save_path}")
    
    plt.show()

print("✅ Visualization functions defined")

## 🔄 Model Loading & Inference Functions

In [ ]:
def load_complete_model(model_path=None):
    """
    Load a saved model with all its components
    """
    
    if model_path is None:
        model_path = os.path.join(MODEL_DIR, 'latest_model.h5')
    
    print(f"\n📂 Loading model from: {model_path}")
    
    # Load model
    model = load_model(model_path)
    print("✅ Model loaded successfully")
    
    # Load scaler
    scaler_path = os.path.join(SCALER_DIR, 'feature_scaler.pkl')
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    print(f"✅ Scaler loaded from: {scaler_path}")
    
    # Load feature names
    feature_path = os.path.join(METADATA_DIR, 'feature_names.json')
    with open(feature_path, 'r') as f:
        feature_names = json.load(f)
    print(f"✅ Feature names loaded from: {feature_path}")
    
    return {
        'model': model,
        'scaler': scaler,
        'feature_names': feature_names
    }

def predict_new_data(model_components, new_data):
    """
    Make predictions on new data
    
    Args:
        model_components: dict with 'model', 'scaler', 'feature_names'
        new_data: DataFrame with features
    """
    
    # Extract required features
    X_new = new_data[model_components['feature_names']]
    
    # Scale features
    X_scaled = model_components['scaler'].transform(X_new)
    
    # Reshape for LSTM
    X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
    
    # Predict
    predictions = model_components['model'].predict(X_lstm, verbose=0).flatten()
    
    return predictions

print("✅ Loading and inference functions defined")

## 🚀 COMPLETE PIPELINE EXECUTION
### Run this section to train and save your model

In [ ]:
"""
MAIN EXECUTION PIPELINE

Uncomment and run this section to:
1. Load your data
2. Create enhanced features
3. Train BiLSTM model
4. Save model in multiple formats
5. Evaluate performance
6. Generate visualizations
"""

# Step 1: Load the dataset
df = pd.read_excel('soya_data_cleaned_2023_onwards.xlsx')

# Convert date columns
date_columns = ['sales_order_creation_date', 'promised_expedition_date', 'actual_expedition_date', 'date_and_time_expedition']
for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print(f"Dataset shape before combining batches: {df.shape}")
print(f"Date range: {df['actual_expedition_date'].min()} to {df['actual_expedition_date'].max()}")

# Combine orders with same client_order_number (delivered in batches)
print(f"\nTotal records before combining batches: {len(df)}")
print(f"Unique client_order_numbers: {df['client_order_number'].nunique()}")

# Check for duplicates
duplicates = df['client_order_number'].duplicated().sum()
print(f"Orders delivered in multiple batches: {duplicates}")

# Group by client_order_number and aggregate:
# - actual_expedition_date: take the LAST date (max) - when final batch was delivered
# - total_amount_delivered_tm: SUM all batches
# - other fields: take first (they should be the same for the same order)
agg_dict = {
    'sales_order_creation_date': 'first',
    'actual_expedition_date': 'max',  # Last delivery date
    'total_amount_delivered_tm': 'sum',  # Sum all batches
    'client_name': 'first',
}

# Add any other columns that exist in the dataframe
for col in df.columns:
    if col not in agg_dict and col != 'client_order_number':
        agg_dict[col] = 'first'

df = df.groupby('client_order_number', as_index=False).agg(agg_dict)

print(f"After combining batches: {len(df)} unique orders")
print()

df.head()

In [ ]:
# Create order size categories
df['order_size_category'] = pd.cut(
    df['total_amount_delivered_tm'],
    bins=[0, 5, 10, 20, np.inf],
    labels=['Small', 'Medium', 'Large', 'Extra Large'],
    include_lowest=True
)

print("Order Size Distribution:")
print(df['order_size_category'].value_counts())
print(f"\nPercentage:")
print(df['order_size_category'].value_counts(normalize=True) * 100)

In [ ]:
# Filter for Small and Medium orders only
df_filtered = df[df['order_size_category'].isin(['Small', 'Medium'])].copy()

print(f"Original dataset: {len(df)} records")
print(f"Filtered dataset (Small + Medium): {len(df_filtered)} records")
print(f"\nCategory breakdown:")
print(df_filtered['order_size_category'].value_counts())

In [ ]:
# Step 2: Create features
df_features = create_enhanced_features(df_filtered)
print("⚠️  Step 2: Create features")

# Step 3: Select features
selected_features = select_best_features()
print("⚠️  Step 3: Select features")

# Step 4: Prepare data
data = prepare_data_for_lstm(df_features, selected_features, CONFIG)
print("⚠️  Step 4: Prepare data")

In [ ]:
# Step 5: Build model
input_shape = (data['X_train'].shape[1], data['X_train'].shape[2])
model = build_enhanced_bilstm(input_shape, CONFIG)
print(model.summary())
print("⚠️  Step 5: Build model")

# Step 6: Create callbacks
callbacks, timestamp = create_callbacks(CONFIG)
print("⚠️  Step 6: Create callbacks")



In [ ]:
# Step 7: Train model
model, history = train_bilstm(model, data, CONFIG, callbacks)
print("⚠️  Step 7: Train model")



In [ ]:
# # Step 8: Save model
# model_name, metadata = save_complete_model(model, data, CONFIG, history, timestamp)
# print("⚠️  Step 8: Save model")



In [ ]:
# Step 9: Evaluate
results = evaluate_model_comprehensive(model, data['X_test'], data['y_test'])
print("⚠️  Step 9: Evaluate model")



In [ ]:
# Step 10: Visualize
plot_training_history(history, save_path='training_history.png')
plot_predictions(data['y_test'], results['predictions'], save_path='predictions.png')
print("⚠️  Step 10: Create visualizations")


## 🔮 Quick Inference Example

In [ ]:
"""
QUICK INFERENCE EXAMPLE

Use this to load a saved model and make predictions
"""

# Load the latest saved model
model_components = load_complete_model()

# Make predictions on new data
predictions = predict_new_data(model_components, new_df)

print(f"Predictions: {predictions}")

print("\n✅ Inference example ready!")
print("\nUncomment to load model and make predictions.")

## 📋 Summary

### Your Current Performance (Excellent! 🎉)
- **MAE: 3.59 days** (Much better than baseline 6.17 days)
- **Within 7 days: 91.9%** (Outstanding!)
- **Within 14 days: 96.3%** (Excellent precision)

### What This Notebook Provides

1. **Enhanced Architecture**
   - Bidirectional LSTM (128 → 64 units)
   - Batch normalization
   - L1/L2 regularization
   - Dropout for regularization

2. **Comprehensive Model Saving**
   - ✅ Full model (.h5)
   - ✅ SavedModel format (TensorFlow)
   - ✅ Weights only
   - ✅ Architecture (JSON)
   - ✅ Training history
   - ✅ Metadata
   - ✅ Scaler
   - ✅ Feature names

3. **Training Features**
   - Early stopping
   - Learning rate reduction
   - Model checkpointing
   - CSV logging
   - TensorBoard support

4. **Easy Inference**
   - Simple model loading
   - Prediction on new data
   - All preprocessing handled automatically

### Next Steps

1. Load your data
2. Run the pipeline
3. Model will be saved automatically
4. Use `load_complete_model()` for inference
5. Share the `models/` directory for deployment

**Your model is already performing excellently! This notebook helps you save and deploy it properly.** 🚀